In [1]:
!pip install sentence_transformers
!git clone https://github.com/RusinDmitry/Identifying-implicit-relationships-in-abstract-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=41b0760e64d778a57806174f1d6cdae3c89676b4b8b579944a6cd9d3e2286507
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
Cloning into 'Identifying-implicit-relationships-in-abstract-text'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 5), reused 3 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 3.66 MiB | 11.32 MiB/s, done.
Resolving deltas: 100% (5/5), done.


Загрузка данных

In [2]:
import pandas as pd


def read_dat_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    data = []
    current_data = []
    for line in lines:
        if line.strip() == "*$*":
            if current_data:
                data.append(current_data)
            current_data = []
        else:
            current_data.append(line.strip())

    if current_data:
        data.append(current_data)

    return data


def create_ngrams(lst, n):
    return [' '.join(lst[i:i+n]) for i in range(len(lst)-n+1)]

file_path = '/content/Identifying-implicit-relationships-in-abstract-text/all_train_df.csv'
data = pd.read_csv(file_path)["Data"].iloc[:]
data = data.apply(lambda x: x.split("\n")).to_list()

## Получение вложений слов

In [24]:
from sentence_transformers import SentenceTransformer, util
import torch

corpus = sum([sum([create_ngrams(data[i], n) for n in range(1, 2)], []) for i in range(len(data))], [])

print(len(corpus))
print(len(pd.Series(corpus).drop_duplicates()))
corpus = pd.Series(corpus).drop_duplicates().to_list()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
model = SentenceTransformer('distiluse-base-multilingual-cased-v1', device = DEVICE)
embeddings = model.encode(corpus, convert_to_tensor=True)


45587
9628
cuda


## Кластеризация

In [25]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

clustering_model = AgglomerativeClustering(n_clusters=100) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(embeddings.cpu())
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(*cluster, sep="\n")
    print("")

Выходные данные были обрезаны до нескольких последних строк (5000).
Номер объекта: CGN-20220614008, идентифицированный ниже составитель проекта:
Номер: CGN-202207010005, в котором определены следующие участники проекта:
Номер: CGN-20210817001, определяющий медианный состав проекта следующим образом:
Проект AD (CGN-202207210007) был синтезирован для экскавации и определен следующим образом:
В рамках проекта LOT134F &lt; &lt; Гуандун &gt; &gt; (заказ No CGN-202108180010) были определены следующие участники проекта:
Закупка строительных работ в Северной Корее (закупка тендера: CGN-202212010005) идентифицирует участников проекта следующим образом:
Проект BJ по проекту &quot; Загрязнитель &quot; (номер тендера: CGN-202211100004) был определен следующим образом:
Приобретение оборудования для системы высокого давления (номер тендера: CGN-202304270011) идентифицирует участников проекта следующим образом:
В проекте CS по водоснабжению (номер тендера: CGN-202210120006) определены следующие участ

## Сохранение

In [34]:
clusters = pd.Series(data = clustered_sentences.keys(), index = clustered_sentences.values())
clusters.to_csv("/content/drive/MyDrive/clusters.csv")